In [45]:
# 14/01/2021 Luís

# A first version of the HGO model.

# Need to check if the function is working accordingly to literature. Then I will irove the function, for example
# by implementing different loads as a function of a string argument of the function.
# For now it only allows a uniaxial load, assuming incompressibility of the material.

# It accepts a list of parameters params = [c, κ, k1, k2, a01, a02], a01 and a02 being 3 dimensional vectors.
# This might not be the best solution, maybe it would be better for the function to accept more arguments
# instead of this list of params.

# Need to get the stretch-strain curves by generating some parameters, like in the other notebooks.

In [46]:
import pandas as pd
import random

import sympy as sym
from sympy.physics.quantum import TensorProduct

import numpy as np
from matplotlib import pyplot as plt

pd.set_option('display.max_rows', None)

In [47]:
def HGO(params, stretch):
    #returns cauchy stress at yy direction
    
    #3x3 Identity Matrix
    I = sym.Matrix([[1,0,0],[0,1,0],[0,0,1]])
    
    # Modified Right Cauchy-Green Deformation Tensor 'Cm'
    Cm11 = sym.Symbol('Cm11')
    Cm12 = sym.Symbol('Cm12')
    Cm13 = sym.Symbol('Cm13')
    Cm21 = sym.Symbol('Cm21')
    Cm22 = sym.Symbol('Cm22')
    Cm23 = sym.Symbol('Cm23')
    Cm31 = sym.Symbol('Cm31')
    Cm32 = sym.Symbol('Cm32')
    Cm33 = sym.Symbol('Cm33')
    Cm = sym.Matrix([[Cm11,Cm12,Cm13], [Cm21,Cm22,Cm23], [Cm31,Cm32,Cm33]])  
    
    # Compute the invariant im1  of the tensor Cm
    im1=sym.trace(Cm)
    
    
    #symbolic Neo-Hookean parameter c
    c = sym.Symbol('c')
    
    # symbolic dispersion parameter κ (0 < κ < 1/3) (the symbol is the greek letter 'kappa')
    κ = sym.Symbol('κ')
    
    # symbolic material parameters k1 and k2 (k1>0; k2>0)
    k1=sym.Symbol('k1')
    k2=sym.Symbol('k2')
    
    # symbolic unit vectors representing the direction of the fibres in the stress free configuration
    a01_1 = sym.Symbol('a01_1')
    a01_2 = sym.Symbol('a01_2')
    a01_3 = sym.Symbol('a01_3')
    a01 = sym.Matrix([a01_1, a01_2, a01_3])
    
    a02_1 = sym.Symbol('a02_1')
    a02_2 = sym.Symbol('a02_2')
    a02_3 = sym.Symbol('a02_3')
    a02 = sym.Matrix([a02_1, a02_2, a02_3])
    
    # Structure Tensors H1 and H2, which depend on κ and a01, a02
    H1 = κ*I + (1-3*κ)*(TensorProduct(a01,sym.transpose(a01)))
    H2 = κ*I + (1-3*κ)*(TensorProduct(a02,sym.transpose(a02)))
    
    E1 = sum(H1.multiply_elementwise(Cm)) - 1
    E2 = sum(H2.multiply_elementwise(Cm)) - 1 
    
    #Generate SEF (Strain Energy Function)
    sef= 0.5 * c * (im1 - 3) + (k1/(2*k2)) * (sym.exp(k2*E1) - 1) + (k1/(2*k2)) * (sym.exp(k2*E2) - 1)
    
    #Second Piola Kirchoff Stresses
    S11=2*sym.diff(sef,Cm11)
    S12=2*sym.diff(sef,Cm12)
    S13=2*sym.diff(sef,Cm13)
    S21=2*sym.diff(sef,Cm21)
    S22=2*sym.diff(sef,Cm22)
    S23=2*sym.diff(sef,Cm23)
    S31=2*sym.diff(sef,Cm31)
    S32=2*sym.diff(sef,Cm32)
    S33=2*sym.diff(sef,Cm33)
    S = sym.Matrix([[S11,S12,S13], [S21,S22,S23], [S31,S32,S33]])
    
    # Deformation Gradient assuming incompressibility and a uniaxial load
    F=sym.Matrix([[1/(np.sqrt(stretch)),0,0], [0,stretch,0], [0,0,1/(np.sqrt(stretch))]])
    Ft=sym.transpose(F)
    
    Jac=sym.det(F)
    
    Fm = Jac**(-1/3) * I * F
    
    Fmt=sym.transpose(Fm)
    
    Cm=Fmt*Fm

    Cm11v=Cm[0,0]
    Cm12v=Cm[0,1]
    Cm13v=Cm[0,2]
    Cm21v=Cm[1,0]
    Cm22v=Cm[1,1]
    Cm23v=Cm[1,2]
    Cm31v=Cm[2,0]
    Cm32v=Cm[2,1]
    Cm33v=Cm[2,2]
    
    cauchy=(1/Jac)*(F*S*Ft) # cauchy stresses with no BCs
    stress=cauchy[0,0]-cauchy[2,2] # imposing of boundary conditions
    
    stress_abq=sym.Matrix([[0,0,0], [0, stress,0], [0,0,0]])
    
    s11_val=stress_abq.subs([(Cm11, Cm11v), (Cm12, Cm12v), 
                                  (Cm13, Cm13v),(Cm21, Cm21v), 
                                  (Cm22, Cm22v), (Cm23, Cm23v),
                                  (Cm31, Cm31v), (Cm32, Cm32v), 
                                  (Cm33, Cm33v),(c,params[0]), 
                                  (κ,params[1]),(k1,params[2]),(k2,params[3]),
                                  (a01_1,params[4][0]),(a01_2,params[4][1]),(a01_3,params[4][2]),
                                  (a02_1,params[4][0]),(a02_2,params[4][1]),(a02_3,params[4][2])])
    
    return stretch, s11_val[1,1]

def get_curve(params,stretch_min,stretch_max,ninc):
    #stores HGO loading runs between a minimum and a maximum stretch
    lst=[HGO(params, stretch) for stretch in np.linspace(stretch_min,stretch_max,ninc)]
    return lst

In [48]:
# Let us aplly our function to an example


c= 7.64 # kPa
κ = 0.226
k1 = 996.6 # kPa
k2 = 524.6
a01 = [-np.sin(np.deg2rad(65)),np.cos(np.deg2rad(65)),0]
a02 = [np.sin(np.deg2rad(65)),np.cos(np.deg2rad(65)),0]


params = [c, κ, k1, k2, a01, a02]
stretch = 1.2

[stretch, stress] = HGO(params, stretch)

stress # in kPa

7196.87557738371